In [ ]:
import pandas as pd
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import random
import time
import re

In [ ]:
if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device("cpu")
device

device(type='cuda')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/train/training.1600000.processed.noemoticon.csv", error_bad_lines=  False, encoding= 'latin-1', header= None)

In [ ]:
eng = []
for i in range(10000):
  eng += [df.at[i, 5]]eng = []

for i in data:
  eng += [i[0]]

In [ ]:
for i in range(len(eng)):
  x = re.sub(r'[^a-zA-Z ]+', '', eng[i])
  x = x.lower()
  x = x.split(" ")
  x = [i for i in x if len(i)>0]
  x = ' '.join(x)
  eng[i] = x

In [ ]:
!git clone "https://github.com/anoopkunchukuttan/indic_nlp_library"

fatal: destination path 'indic_nlp_library' already exists and is not an empty directory.


In [ ]:
!git clone https://github.com/anoopkunchukuttan/indic_nlp_resources.git

fatal: destination path 'indic_nlp_resources' already exists and is not an empty directory.


In [ ]:
!pip install Morfessor

In [ ]:
INDIC_NLP_LIB_HOME=r"/content/indic_nlp_library"
INDIC_NLP_RESOURCES="/content/indic_nlp_resources"

import sys
sys.path.append(r'{}'.format(INDIC_NLP_LIB_HOME))

from indicnlp import common
common.set_resources_path(INDIC_NLP_RESOURCES)

from indicnlp import loader
loader.load()

from indicnlp.normalize.indic_normalize import BaseNormalizer
from indicnlp.tokenize import indic_tokenize

normalizer = BaseNormalizer("hi", remove_nuktas=True)

In [ ]:
eng2tok={'<sos>':0, '<eos>':1}
tok2eng={0:'<sos>', 1:'<eos>'}
cnt_eng=2
def addwordeng(word):
  global cnt_eng, eng2tok, tok2eng;
  if word not in eng2tok:
    eng2tok[word]=cnt_eng
    tok2eng[cnt_eng]=word
    cnt_eng+=1

In [ ]:
src=[]
for i in range(1, len(eng)):
  x = indic_tokenize.trivial_tokenize(eng[i])
  for j in x:
    addwordeng(j)
  x.insert(0, '<sos>')
  x.append('<eos>')
  src += [x]

In [ ]:
for i in range(0,len(src)):
  if len(src[i])>25:
    src[i]=src[i][:24]
    src[i].append('<eos>')

In [ ]:
# tokenized text is numericalized7.  You can cite papers like this [1, 2].  You can create a footnote like this
src_tok=[]
for i in src:
  src_tok+=[[eng2tok[j] for j in i]]

trg_tok=[]
for i in src:
  trg_tok+=[[eng2tok[j] for j in i]]

In [ ]:
class Encoder(nn.Module):
  def __init__(self, input_size, hidden_dim, embedding_dim):
    super().__init__()
    self.hidden_dim = hidden_dim
    self.embedding_dim = embedding_dim
    self.embedding = nn.Embedding(input_size, embedding_dim)
    self.GRU = nn.GRU(embedding_dim, hidden_dim)

  def forward(self, input_tensor, hidden):
    output_tensor = self.embedding(input_tensor).view(1, 1, -1)
    output_tensor, hidden = self.GRU(output_tensor, hidden)
    return output_tensor, hidden

  def initHidden(self):
    x = torch.zeros(1, 1, self.hidden_dim, device=device)
    return x

In [ ]:
class Decoder(nn.Module):
  def __init__(self, output_size, hidden_dim, embedding_dim):
    super().__init__()
    self.hidden_dim = hidden_dim
    self.embedding_dim = embedding_dim
    self.embedding = nn.Embedding(output_size, embedding_dim)
    self.GRU = nn.GRU(embedding_dim, hidden_dim)
    self.out = nn.Linear(hidden_dim, output_size)
    self.softmax = nn.LogSoftmax(dim=1)

  def forward(self, input_tensor, hidden):
    output_tensor = self.embedding(input_tensor).view(1, 1, -1)
    output_tensor = F.relu(output_tensor)
    output_tensor, hidden = self.GRU(output_tensor, hidden)
    output_tensor = self.out(output_tensor[0])
    output_tensor = self.softmax(output_tensor)
    return output_tensor, hidden

  def initHidden(self):
    x = torch.zeros(1, 1, self.hidden_size, device=device)
    return x

In [ ]:
def train(encoder, encoder_optimizer, decoder, decoder_optimizer, criteria, src, trg):
  encoder_optimizer.zero_grad()
  decoder_optimizer.zero_grad()

  encoder_hidden = encoder.initHidden()

  loss = 0
  teacher_forcing = 0.4

  for i in range(src.size(0)):   # iterate the input length
    encoder_output, encoder_hidden = encoder(src[i], encoder_hidden)

  decoder_input = torch.tensor([[0]], device=device)
  decoder_hidden = encoder_hidden

  if random.random() < teacher_forcing:
    for i in range(trg.size(0)):  # iterate the input length
      decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
      loss += criteria(decoder_output, trg[i])
      decoder_input = trg[i]  # Teacher forcing

  else:
    for i in range(trg.size(0)):  # iterate the input length
      decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
      topv, topi = decoder_output.topk(1)
      decoder_input = topi.squeeze().detach()  # detach from history as input

      loss += criteria(decoder_output, trg[i])
      if decoder_input.item() == 1: # if EOS token is encountered
        break

  loss.backward()

  encoder_optimizer.step()
  decoder_optimizer.step()

  return loss.item() / trg.size(0)

In [ ]:
# run this cell if you want to continue traing the model

# encoder = Encoder(46120, 512, 256).to(device) # (dict_size, hidden_dim, embedding_dim)
# decoder = Decoder(37706, 512, 256).to(device)

# checkpoint = torch.load('/content/drive/MyDrive/parameter.pt')
# encoder.load_state_dict(checkpoint['modelA_state_dict'])
# decoder.load_state_dict(checkpoint['modelB_state_dict'])

In [ ]:
# uncomment the encoder and decoder to initialize the encoder-decoder architecture and start training from scratch

encoder = Encoder(17120, 64, 64).to(device) # (dict_size, hidden_dim, embedding_dim)
decoder = Decoder(17120, 64, 64).to(device)
encoder.train()
decoder.train()

epoch = 10       # total = 5

start = time.time()
total_loss = 0
glob_loss = 0
num_iters = 9999 * epoch
batch =500

encoder_optim = optim.SGD(encoder.parameters(), lr=0.01)
decoder_optim = optim.SGD(decoder.parameters(), lr=0.01)

criteria = nn.NLLLoss()

input = [torch.tensor(i, dtype=torch.long, device=device).view(-1, 1) for i in src_tok]
target = [torch.tensor(i, dtype=torch.long, device=device).view(-1, 1) for i in trg_tok]

for i in range(1, num_iters + 1):
  src = input[(i-1)%9999]
  trg = target[(i-1)%9999]

  loss = train(encoder, encoder_optim, decoder, decoder_optim, criteria, src, trg)
  total_loss += loss
  glob_loss  += loss

  if i % batch == 0:
    avg_loss= round(total_loss / batch, 4)
    total_loss = 0
    end = time.time()
    mins = int((end - start) / 60)
    secs = int((end-start) % 60)
    time_taken = str(mins) + "m " + str(secs) + "s"
    curr_epoch = (i // 9999) + 1
    curr_batch = ((i % 9999) // batch)
    print("time: " + time_taken + " Epoch:" + str(curr_epoch) + " Batch:" + str(curr_batch) + "/" + str(9999//batch) + " Loss:" + str(avg_loss))

  if i % 9999 == 0:
    print("Epoch Completed. Epoch Loss:" + str(round(glob_loss/ 9999, 4)))
    print("-----------------------------------------------------------------------------------------------------------------------------------------")
    glob_loss = 0

time: 0m 15s Epoch:1 Batch:1/19 Loss:4.7365
time: 0m 30s Epoch:1 Batch:2/19 Loss:4.5906
time: 0m 46s Epoch:1 Batch:3/19 Loss:4.6462
time: 1m 1s Epoch:1 Batch:4/19 Loss:4.3857
time: 1m 16s Epoch:1 Batch:5/19 Loss:4.4298
time: 1m 32s Epoch:1 Batch:6/19 Loss:4.5854
time: 1m 47s Epoch:1 Batch:7/19 Loss:4.4342
time: 2m 3s Epoch:1 Batch:8/19 Loss:4.491
time: 2m 19s Epoch:1 Batch:9/19 Loss:4.4306
time: 2m 34s Epoch:1 Batch:10/19 Loss:4.3927
time: 2m 50s Epoch:1 Batch:11/19 Loss:4.5464
time: 3m 6s Epoch:1 Batch:12/19 Loss:4.6049
time: 3m 21s Epoch:1 Batch:13/19 Loss:4.7085
time: 3m 39s Epoch:1 Batch:14/19 Loss:5.0303
time: 3m 56s Epoch:1 Batch:15/19 Loss:4.9933
time: 4m 14s Epoch:1 Batch:16/19 Loss:4.999
time: 4m 31s Epoch:1 Batch:17/19 Loss:5.0377
time: 4m 49s Epoch:1 Batch:18/19 Loss:5.0493
time: 5m 7s Epoch:1 Batch:19/19 Loss:5.076
Epoch Completed. Epoch Loss:4.7094
------------------------------------------------------------------------------------------------------------------------------

In [ ]:
  # saving the model for the purpose of re-training

torch.save({
            'modelA_state_dict': encoder.state_dict(),
            'modelB_state_dict': decoder.state_dict(),
            'optimizerA_state_dict': encoder_optim,
            'optimizerB_state_dict': decoder_optim
            }, '/content/drive/MyDrive/autoencoder.pt')

In [ ]:
def evaluate(encoder, decoder, input_tensor):
  encoder.eval()
  decoder.eval()
  output = []

  with torch.no_grad():
    input_length = input_tensor.size()[0]

    encoder_h_t = encoder.initHidden()
    encoder_c_t = encoder.initHidden()

    for i in range(input_length):   # iterate the input length
      encoder_output, encoder_h_t, encoder_c_t = encoder(input_tensor[i], encoder_h_t, encoder_c_t)

    return encoder_output